# Temperature Exploration

March 5, 2023  
Gully & Ryan H.

Using Fleck and Batman to create cartoons showing how starspots effect the flux of stars.

In [ ]:
from batman import TransitParams
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u
from fleck import Star
import lightkurve as lk

## Example from website:

In [ ]:
planet = TransitParams() # we are using a planet to simulate a starspot since they act in a similar manner
planet.per = 88 # orbital period
planet.a = float(0.387*u.AU / u.R_sun) # Semi-major axis [in stellar radii].
planet.rp = 0.1 # Planet radius [in stellar radii].
planet.w = 90 # Argument of periapse [in degrees]
planet.ecc = 0 # Orbital eccentricity.
planet.inc = 89.65 # Orbital inclination [in degrees].
planet.t0 = 0 # Time of inferior conjunction.
planet.limb_dark = 'quadratic' # Limb darkening model
planet.u = [0.5079, 0.2239] # List of limb darkening coefficients.
planet.lam = 45

In [ ]:
inc_stellar = 70 * u.deg
spot_radii = np.array([[0.1], [0.1]])
spot_lons = np.array([[0], [0]]) * u.deg
spot_lats = np.array([[25], [-25]]) * u.deg

In [ ]:
times = np.linspace(-0.5, 0.5, 500)

In [ ]:
star = Star(spot_contrast=0.7, u_ld=planet.u, rotation_period=10)

In [ ]:
lc = star.light_curve(spot_lons, spot_lats, spot_radii,
                      inc_stellar, planet=planet, times=times)

In [ ]:
plt.plot(times, lc, color='k')
plt.show()

In [ ]:
ax = star.plot(spot_lons, spot_lats, spot_radii, inc_stellar, planet=planet, time=0)
plt.show()

## Testing:

In [ ]:
# Choose random ligthcurve to model in Fleck:
df = pd.read_csv('../../data/Rapid_Rotator_Sample.csv')
random = df.iloc[2230].EPIC
print(random)
sr = lk.search_lightcurve(f"EPIC {random}", mission='TESS')
lc = sr[0].download().remove_nans().remove_outliers().normalize()#.flatten()
scalar = np.nanpercentile(lc.flux, 98)
lc = lc/scalar
lc.plot()
plt.xlim(1573, 1579.5)
plt.ylim(0.965, 1.01)
plt.show()

In [ ]:
# planet = TransitParams() # we are using a planet to simulate a starspot since they act in a similar manner
# planet.per = 1000 # orbital period
# planet.a = float(0.29*u.AU / u.R_sun) # Semi-major axis [in stellar radii].
# planet.rp = 0.15 # Planet radius [in stellar radii].
# planet.w = 80 # Argument of periapse [in degrees]
# planet.ecc = 0.1 # Orbital eccentricity.
# planet.inc = 90 # Orbital inclination [in degrees].
# planet.t0 = 0 # Time of inferior conjunction.
# planet.limb_dark = 'quadratic' # Limb darkening model
# planet.u = [0.5079, 0.2239] # List of limb darkening coefficients.
# planet.lam = 0

planet = TransitParams()
planet.t0 = 1346.5    # transit mid-time
planet.per = 3.4567  # orbital period
planet.rp = 0.02     # planet-to-star radius ratio
planet.a = 20        # semi-major axis (in units of stellar radius)
planet.inc = 89.9    # orbital inclination (in degrees)
planet.ecc = 0.      # eccentricity
planet.w = 90.       # longitude of periastron (in degrees)
planet.u = [0.1, 0.3] # limb darkening coefficients
planet.limb_dark = 'quadratic' # Limb darkening model

inc_stellar = 45 * u.deg
spot_radii = np.array([[0.1], [0.1]])
spot_lats = np.array([[90], [0]]) * u.deg
spot_lons = np.array([[360-30], [30]]) * u.deg


times = np.linspace(-3, 3, 500)

star = Star(spot_contrast=0.7, u_ld=planet.u, rotation_period=10)

lc = star.light_curve(spot_lons, spot_lats, spot_radii,
                      inc_stellar, planet=planet, times=times)


plt.plot(times, lc, color='k')
plt.ylabel("Flux [$\mathrm{e}^{-1} \mathrm{s}^{-1}$]")
plt.xlabel("Period [d]")
plt.title("Lightcurve Model")
plt.ylim(0.965, 1.01)
plt.show()

In [ ]:
ax = star.plot(spot_lons, spot_lats, spot_radii, inc_stellar, planet=planet, time=0)
plt.ylabel("Flux [$\mathrm{e}^{-1} \mathrm{s}^{-1}$]")
plt.xlabel("Period [d]")
plt.title("Star Model")
plt.show()

In [ ]:
# Load a Kepler light curve from Lightkurve
lc = lk.search_lightcurvefile('KIC 8462852', quarter=4).download().PDCSAP_FLUX.remove_nans()

# Extract the time and flux values from the light curve
time = lc.time.value
flux = lc.flux.value

# Normalize the flux
flux = flux / np.median(flux)

# Define parameters of the system
params = batman.TransitParams()
params.t0 = time[np.argmin(flux)]  # time of inferior conjunction
params.per = 10.0  # period in days
params.rp = 0.05  # planet radius (in units of stellar radius)
params.a = 20.0  # semi-major axis (in units of stellar radius)
params.inc = 90.0  # inclination angle
params.ecc = 0.0  # eccentricity
params.w = 90.0  # longitude of periastron (in degrees)
params.u = [0.4, 0.2]  # limb darkening coefficients
params.limb_dark = 'quadratic'  # limb darkening model

# Create a batman transit model
t = np.linspace(time.min(), time.max(), 10000)
m = batman.TransitModel(params, t)
model_flux = m.light_curve(params)

# Plot the light curve and model
plt.plot(time, flux, 'k.', markersize=2)
plt.plot(t, model_flux, 'r-', linewidth=2)
plt.xlabel('Time [days]')
plt.ylabel('Normalized Flux')
plt.show()


In [ ]:
inc_stellar = 45 * u.deg
spot_radii = np.array([[0.1], [0.1]])
spot_lats = np.array([[90], [0]]) * u.deg
spot_lons = np.array([[360-30], [30]]) * u.deg
ax = star.plot(spot_lons, spot_lats, spot_radii, inc_stellar, planet=params, time=0)
plt.ylabel("Flux [$\mathrm{e}^{-1} \mathrm{s}^{-1}$]")
plt.xlabel("Period [d]")
plt.title("Star Model")
plt.show()